In [1]:
import numpy as np
import tensorly as tl
from tensorly.decomposition import parafac

# TensorlyのデフォルトバックエンドをNumPyに設定
tl.set_backend('numpy')

# ランダムなテンソルを生成
shape = (4, 5, 6)  # テンソルのサイズ
rank = 3  # 分解するランク

np.random.seed(42)  # 再現性のためのシード設定
tensor = np.random.random(shape)

# マスクを生成 (0または1の値で、一部の要素をマスク)
mask = np.random.choice([0, 1], size=shape, p=[0.2, 0.8])  # 20%の要素をマスク

# マスクを適用したテンソル
masked_tensor = tensor * mask

# CP分解 (マスクを利用)
weights, factors = parafac(masked_tensor, rank=rank, mask=mask, init='random', n_iter_max=100, tol=1e-6)

# 分解結果を表示
print("Weights:")
print(weights)

print("\nFactors:")
for i, factor in enumerate(factors):
    print(f"Factor {i+1}:")
    print(factor)

# 分解の再構成 (元のテンソルに近似)
reconstructed_tensor = tl.cp_to_tensor((weights, factors))

# 再構成エラーの計算 (元のテンソルと比較)
error = tl.norm(tensor - reconstructed_tensor) / tl.norm(tensor)
print(f"\nReconstruction Error: {error}")

# マスクされた部分のエラーを確認
masked_error = tl.norm((tensor - reconstructed_tensor) * mask) / tl.norm(masked_tensor)
print(f"Reconstruction Error on Masked Elements: {masked_error}")


Weights:
[1. 1. 1.]

Factors:
Factor 1:
[[ 0.47008366 -0.30226962  0.64025411]
 [ 1.07388664 -1.01930617  0.43750138]
 [ 0.65376409 -0.76306421  0.77345797]
 [ 0.63995897  0.01913157  0.15154783]]
Factor 2:
[[ 1.12026376  0.56534419  0.67005752]
 [ 0.52967742  0.52553227  1.16173857]
 [ 0.71505516 -0.06386666  0.23932377]
 [ 0.92412083  0.21705376 -0.0551336 ]
 [ 1.12035059  2.7010547   1.09500251]]
Factor 3:
[[ 0.41797102 -0.03492461  0.04151004]
 [ 0.69707907  1.76262657  1.42243205]
 [ 1.17198762  0.45909995  0.01116103]
 [ 0.55456293  0.29606614  0.82605964]
 [ 1.02091975  0.36507947 -0.40571643]
 [ 0.91310598  0.85519094  0.5059477 ]]

Reconstruction Error: 0.9916848792192284
Reconstruction Error on Masked Elements: 0.3601148612256122


In [ ]:
import numpy as np
import tensorly as tl
from tensorly.decomposition import parafac

# TensorlyのデフォルトバックエンドをNumPyに設定
tl.set_backend('numpy')

# ランダムなテンソルを生成
shape = (4, 5, 6)  # テンソルのサイズ
rank = 3  # 分解するランク

np.random.seed(42)  # 再現性のためのシード設定
tensor = np.random.random(shape)

# マスクを生成 (唯一のエントリ以外を全てマスク)
mask = np.zeros(shape, dtype=int)  # 全てマスクされた状態
single_entry_index = (1, 2, 3)  # 利用可能な唯一のエントリのインデックス
mask[single_entry_index] = 1  # この位置だけマスク解除

# マスクを適用したテンソル
masked_tensor = tensor * mask

# CP分解 (マスクを利用)
weights, factors = parafac(masked_tensor, rank=rank, mask=mask, init='random', n_iter_max=100, tol=1e-6)

# 分解結果を表示
print("Weights:")
print(weights)

print("\nFactors:")
for i, factor in enumerate(factors):
    print(f"Factor {i+1}:")
    print(factor)

# 分解の再構成 (元のテンソルに近似)
reconstructed_tensor = tl.cp_to_tensor((weights, factors))

# 再構成エラーの計算
error = tl.norm(tensor - reconstructed_tensor) / tl.norm(tensor)
print(f"\nReconstruction Error: {error}")

# マスクされた部分のエラーを確認
masked_error = tl.norm((tensor - reconstructed_tensor) * mask) / tl.norm(masked_tensor)
print(f"Reconstruction Error on Masked Elements: {masked_error}")


LinAlgError: Singular matrix

In [3]:
import numpy as np
import tensorly as tl
from tensorly.decomposition import parafac
from scipy.stats import qmc

# TensorlyのデフォルトバックエンドをNumPyに設定
tl.set_backend('numpy')

# ランダムなテンソルを生成 (形状を指定)
shape = (10, 10, 10)  # テンソルの形状
rank = 3  # 分解するランク
np.random.seed(42)  # 再現性のためのシード

# Sobol列を生成して、全体の10%を選択
sampler = qmc.Sobol(d=len(shape), scramble=True)
n_samples = int(np.prod(shape) * 0.1)  # 全体の10%
samples = sampler.random_base2(m=int(np.ceil(np.log2(n_samples))))
indices = (samples * np.array(shape)).astype(int)

# マスクを生成 (全てマスク済みに初期化)
mask = np.zeros(shape, dtype=int)

# Sobol列で選んだインデックスに1を設定 (利用可能な部分)
for idx in indices:
    mask[tuple(idx)] = 1

# 1つのエントリを適当に選んで固定
fixed_entry_index = (5, 5, 5)  # 適当に指定
mask[fixed_entry_index] = 1

# 元テンソルを標準正規分布で埋める
tensor = np.random.normal(loc=0.0, scale=1.0, size=shape)

# マスクを適用して観測可能部分を設定
masked_tensor = tensor * mask

# CP分解 (マスクを利用)
weights, factors = parafac(masked_tensor, rank=rank, mask=mask, init='random', n_iter_max=100, tol=1e-6)

# 分解結果を表示
print("Weights:")
print(weights)

print("\nFactors:")
for i, factor in enumerate(factors):
    print(f"Factor {i+1}:")
    print(factor)

# 分解の再構成 (元のテンソルに近似)
reconstructed_tensor = tl.cp_to_tensor((weights, factors))

# 再構成エラーの計算
error = tl.norm(tensor - reconstructed_tensor) / tl.norm(tensor)
print(f"\nReconstruction Error: {error}")

# マスクされた部分のエラーを確認
masked_error = tl.norm((tensor - reconstructed_tensor) * mask) / tl.norm(masked_tensor)
print(f"Reconstruction Error on Masked Elements: {masked_error}")


Weights:
[1. 1. 1.]

Factors:
Factor 1:
[[-0.84989863  1.38559007 -0.43244416]
 [-1.16164309  0.37901377  1.05986109]
 [-1.80632543  1.25045191  1.42006378]
 [ 0.41208396  0.80044591 -1.09033536]
 [ 0.53728887 -1.31899654  0.71488184]
 [-0.31219434  0.42964377 -1.02145283]
 [-0.92643822  0.48334332  1.13994122]
 [-2.20408489  1.48358025 -0.17329865]
 [ 0.23257241 -0.39028858  0.95263085]
 [ 2.01587509 -0.12556513 -2.04820826]]
Factor 2:
[[-1.16159281  0.94613241  0.07827171]
 [ 0.00707225  0.56960404  0.02444541]
 [ 0.61054596  0.92305871  1.50313692]
 [-0.47289793 -1.00163057  0.69299419]
 [-0.88593792 -1.32190716  0.15369445]
 [ 0.70322942  1.10461717  0.30207419]
 [ 1.51813671  0.26134319  1.27864742]
 [-0.80777833 -0.66228136 -0.60587042]
 [ 0.31097708  0.18432412  0.20122854]
 [-0.64695047 -1.29128199  0.25356829]]
Factor 3:
[[ 0.20221665  0.74345583 -0.53837456]
 [ 0.10969706  0.16064769 -0.8981499 ]
 [ 0.46511042  0.28305495  0.71356606]
 [ 0.88773885  1.57489229  0.34311951]
 [